In [2]:
import pandas as pd
from datasets import load_dataset
import matplotlib.pyplot as plt
import ast

#Loading data
dataset = load_dataset('lukebarousse/data_jobs')
df = dataset['train'].to_pandas()

#data cleaning
df['job_posted_date'] = pd.to_datetime(df['job_posted_date'])

In [3]:
df.head()

,job_title_short,job_title,job_location,job_via,job_schedule_type,job_work_from_home,search_location,job_posted_date,job_no_degree_mention,job_health_insurance,job_country,salary_rate,salary_year_avg,salary_hour_avg,company_name,job_skills,job_type_skills
0,Senior Data Engineer,Senior Clinical Data Engineer / Principal Clin...,"Watertown, CT",via Work Nearby,Full-time,False,"Texas, United States",2023-06-16 13:44:15,False,False,United States,None,NaN,NaN,Boehringer Ingelheim,None,None
1,Data Analyst,Data Analyst,"Guadalajara, Jalisco, Mexico",via BeBee México,Full-time,False,Mexico,2023-01-14 13:18:07,False,False,Mexico,None,NaN,NaN,Hewlett Packard Enterprise,"['r', 'python', 'sql', 'nosql', 'power bi', 't...","{'analyst_tools': ['power bi', 'tableau'], 'pr..."
2,Data Engineer,"Data Engineer/Scientist/Analyst, Mid or Senior...","Berlin, Germany",via LinkedIn,Full-time,False,Germany,2023-10-10 13:14:55,False,False,Germany,None,NaN,NaN,ALPHA Augmented Services,"['python', 'sql', 'c#', 'azure', 'airflow', 'd...","{'analyst_tools': ['dax'], 'cloud': ['azure'],..."
3,Data Engineer,LEAD ENGINEER - PRINCIPAL ANALYST - PRINCIPAL ...,"San Antonio, TX",via Diversity.com,Full-time,False,"Texas, United States",2023-07-04 13:01:41,True,False,United States,None,NaN,NaN,Southwest Research Institute,"['python', 'c++', 'java', 'matlab', 'aws', 'te...","{'cloud': ['aws'], 'libraries': ['tensorflow',..."
4,Data Engineer,Data Engineer- Sr Jobs,"Washington, DC",via Clearance Jobs,Full-time,False,Sudan,2023-08-07 14:29:36,False,False,Sudan,None,NaN,NaN,Kristina Daniel,"['bash', 'python', 'oracle', 'aws', 'ansible',...","{'cloud': ['oracle', 'aws'], 'other': ['ansibl..."


In [4]:
filtered = df[df['salary_year_avg'].notna()].copy()
filtered.reset_index(inplace=True)
filtered['salary_year_avg']

0        109500.0
1        140000.0
2        120000.0
3        228222.0
4         89000.0
           ...   
21998    139216.0
21999    150000.0
22000    221875.0
22001    157500.0
22002    157500.0
Name: salary_year_avg, Length: 22003, dtype: float64

In [5]:
def projected_salary(salary):
    return salary * 1.03

#filtered['salary_year_inflation'] = filtered['salary_year_avg'].apply(projected_salary)
#filtered['salary_year_inflation'] = filtered['salary_year_avg'] * 1.03
filtered['salary_year_inflation'] = filtered['salary_year_avg'].apply(lambda salary: salary * 1.03)
filtered[['salary_year_avg', 'salary_year_inflation']]

,salary_year_avg,salary_year_inflation
0,109500.0,112785.00
1,140000.0,144200.00
2,120000.0,123600.00
3,228222.0,235068.66
4,89000.0,91670.00
...,...,...
21998,139216.0,143392.48
21999,150000.0,154500.00
22000,221875.0,228531.25
22001,157500.0,162225.00


In [6]:
type(ast.literal_eval(df['job_skills'][1]))

list

In [7]:
def cleanList(skills):
    if(pd.notna(skills)):
        return ast.literal_eval(skills)

df['job_skills'] = df['job_skills'].apply(cleanList)

In [8]:
df['job_skills'][0]

In [9]:
# using lambda function
filtered['job_skills'] = filtered['job_skills'].apply(lambda skills : ast.literal_eval(skills) if pd.notna(skills) else skills)

In [10]:
type(filtered['job_skills'][0])

list

# Calculate projected salary next year

* Senior roles assume 5%
* Other roles assume 3%

In [11]:
def projected_salary(row):
    if 'Senior' in row['job_title_short']:
        return 1.05 * row['salary_year_avg']
    else:
        return 1.03 * row['salary_year_avg']

filtered['salary_year_inflation'] = filtered.apply(projected_salary, axis=1)
filtered[['job_title_short', 'salary_year_avg', 'salary_year_inflation']]

,job_title_short,salary_year_avg,salary_year_inflation
0,Data Scientist,109500.0,112785.00
1,Data Engineer,140000.0,144200.00
2,Data Engineer,120000.0,123600.00
3,Data Scientist,228222.0,235068.66
4,Data Analyst,89000.0,91670.00
...,...,...,...
21998,Data Engineer,139216.0,143392.48
21999,Data Engineer,150000.0,154500.00
22000,Data Scientist,221875.0,228531.25
22001,Data Scientist,157500.0,162225.00


In [12]:
filtered['salary_year_inflation'] = filtered.apply(
    lambda row: 1.05*row['salary_year_avg'] 
    if 'Senior' in row['job_title_short'] 
    else 1.03*row['salary_year_avg'],
    axis=1
    )

filtered[['job_title_short', 'salary_year_avg', 'salary_year_inflation']]

,job_title_short,salary_year_avg,salary_year_inflation
0,Data Scientist,109500.0,112785.00
1,Data Engineer,140000.0,144200.00
2,Data Engineer,120000.0,123600.00
3,Data Scientist,228222.0,235068.66
4,Data Analyst,89000.0,91670.00
...,...,...,...
21998,Data Engineer,139216.0,143392.48
21999,Data Engineer,150000.0,154500.00
22000,Data Scientist,221875.0,228531.25
22001,Data Scientist,157500.0,162225.00
